A CSV file containing cleaned Uber data can be found here: 

https://drive.google.com/file/d/1pWGv84eaEZF495H7HVwDOFHnLj66-Nbh/view?usp=sharing

In [2]:
import pandas as pd

**Load the local csv files into dataframes.**

**"bus.csv" and "times.csv" should be in the GitHub repo, but uber.csv is not. Find that here: https://drive.google.com/file/d/1QaT4NrrkulKCc1OKrVxXukOJdUp8XiC8/view?usp=sharing**

In [4]:
uber = pd.read_csv('uber.csv', index_col = 0)

C:\Users\theor\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
busDf = pd.read_csv('bus.csv', index_col = 0)

In [6]:
timesDf = pd.read_csv("times.csv", names=["start", "stop", "time"])

**Create a list of unique routes from the busDf**

In [7]:
lines = busDf.ROUTE_OR_LINE.unique()
lines.tolist()

['100',
 '171',
 '225',
 '201',
 '230',
 '60',
 '62',
 '65',
 '78',
 '104',
 '215',
 '26',
 '411',
 '442',
 '449',
 '88',
 'SL5',
 '4',
 '132',
 '170',
 '424',
 '426',
 '434',
 '7',
 '106',
 '35',
 '43',
 '44',
 '51',
 '57A',
 '76',
 'CT3',
 'SL1',
 '111',
 '114',
 '9',
 '116',
 '222',
 '236',
 '326',
 '39',
 '435',
 '119',
 '21',
 '216',
 '428',
 '451',
 '456',
 '459',
 '503',
 '80',
 '87',
 'SL2',
 '8',
 '137',
 '14',
 '220',
 '24',
 '351',
 '554',
 '91',
 '99',
 '501',
 '221',
 '431',
 '448',
 '553',
 '69',
 '92',
 '455',
 '68',
 '746',
 '75',
 '5',
 '11',
 '112',
 '16',
 '22',
 '34',
 '430',
 '502',
 '89',
 '90',
 '109',
 '192',
 '194',
 '214',
 '240',
 '245',
 '441',
 '55',
 '77',
 '117',
 '131',
 '17',
 '211',
 '350',
 '36',
 '436',
 '10',
 '105',
 '108',
 '23',
 '29',
 '31',
 '32',
 '352',
 '45',
 '47',
 '93',
 '210',
 '465',
 '505',
 '71',
 '79',
 '59',
 '72',
 '136',
 '504',
 '558',
 '70',
 '86',
 '202',
 '439',
 '50',
 '57',
 '84',
 '19',
 '30',
 '85',
 'CT1',
 '101',
 '217',

**Sort the Uber dataframe by source and then destination**

In [8]:
uber = uber.sort_values(['sourceid', 'dstid'])

**Function to find the days where every line was slower than 80% of the average.**

If we just look at days where the lines were slower than 100% of the average, we get way too many days to look at. We need to be more strict with out analysis.**

In [9]:
thisLine = busDf.loc[busDf['ROUTE_OR_LINE'] == '101']
averageOnTime = ((thisLine['PERCENT_ONTIME']).mean())

slowerThanAverage = thisLine.loc[(thisLine['PERCENT_ONTIME'] < averageOnTime)]

len(slowerThanAverage)

while (len(slowerThanAverage) > 5):
    averageOnTime *= 0.95
    slowerThanAverage = thisLine.loc[(thisLine['PERCENT_ONTIME'] < averageOnTime)]
    if (slowerThanAverage['PERCENT_ONTIME'].mean() == 0):
        break


In [8]:
slowerThanAverage

,SERVICE_DATE,PEAK_OFFPEAK_IND,MODE_TYPE,ROUTE_OR_LINE,ROUTE_TYPE,STOP,METRIC_TYPE,OTP_NUMERATOR,OTP_DENOMINATOR,PERCENT_ONTIME
124744,6/4/17 0:00,Off-Peak Service (All Other Times),Bus,101,OTH,NaN,Headway / Schedule Adherence,102,253,0.403162
128591,6/18/17 0:00,Off-Peak Service (All Other Times),Bus,101,OTH,NaN,Headway / Schedule Adherence,96,273,0.351648
153125,9/17/17 0:00,Off-Peak Service (All Other Times),Bus,101,OTH,NaN,Headway / Schedule Adherence,93,252,0.369048
183535,1/8/18 0:00,"Peak Service (Weekdays 6:30-9:30AM, 3:30PM-6:3...",Bus,101,OTH,NaN,Headway / Schedule Adherence,105,264,0.397727


In [72]:
def lateDays(lines, busDf):
    
    linesSlowerThanAvg = {}
    count = 0
    tofrom = [[888, 362, 1],[530,885, 7],[1195,888, 15],[888,512, 22],[512,888, 23],[249,888, 28],[525,875,32],[525,1116,39],[1116,810,57],[362,888,66],[402,1129,71],[38,1129,73],[1129,1238,77],[587,837,100],[1116,33,111],[41,254,116]]
    keyLines = ["1", "7", "15", "22", "23", "28", "32", "39", "57", "66", "71", "73", "77", "100", "111", "116"]
    for line in lines:
        #get the piece of the dataframe that has data for this line
        thisLine = busDf.loc[busDf['ROUTE_OR_LINE'] == line]
        
        #find the average percent on time for this line
        averageOnTime = ((thisLine['PERCENT_ONTIME']).mean())
        
        #we want just a handful of delays that are significant relative to each line
        #looking at the averages, we get between 1-100 delays
        #let's make it a max of 5 for each line
        
        slowerThanAverage = thisLine.loc[(thisLine['PERCENT_ONTIME'] < averageOnTime)]
        
        while (len(slowerThanAverage) > 25):
            averageOnTime *= 0.999
            slowerThanAverage = thisLine.loc[(thisLine['PERCENT_ONTIME'] < averageOnTime)]
            if (slowerThanAverage['PERCENT_ONTIME'].mean() == 0):
                break
        
        datesSlowerThanAverage = slowerThanAverage['SERVICE_DATE']
        
        percentOnTime = slowerThanAverage['PERCENT_ONTIME']
        
        listDates = datesSlowerThanAverage.tolist()
        
        listPercents = percentOnTime.tolist()
        count = 0
        for i in tofrom:
            if(line == str(i[2])):
                lcheck = True
                break
            else:
                count = count + 1
        concatenated = [] 
        if(lcheck == True):
        
            for ii in range(len(listDates)):
                concatenated.append((line,tofrom[count][0],tofrom[count][1],listDates[ii], listPercents[ii]))
                concatenated.append((line,tofrom[count][1],tofrom[count][0],listDates[ii], listPercents[ii]))
            #count = count + 1
            print(count)
        else:
            for ii in range(len(listDates)):
                concatenated.append((line,listDates[ii], listPercents[ii]))
        lcheck = False
        linesSlowerThanAvg[line] = concatenated
        
    return linesSlowerThanAvg

Get a dictionary of days that each line was pretty late, including what percent of buses were on-time.

In [73]:
daysSlowerThanAvg = lateDays(lines, busDf)

13
1
14
15
7
3
12
4
6
10
8
11
0
2
5
9


In [11]:
#checking how many values each key has
for key, value in daysSlowerThanAvg.items():
    #print value
    print(key, len([item for item in value if item]))

100 5
171 43
225 5
201 5
230 5
60 5
62 5
65 5
78 5
104 5
215 5
26 5
411 5
442 5
449 20
88 5
SL5 4
4 37
132 5
170 58
424 5
426 5
434 7
7 4
106 5
35 5
43 5
44 3
51 3
57A 5
76 5
CT3 5
SL1 4
111 5
114 23
9 3
116 5
222 4
236 5
326 0
39 4
435 5
119 5
21 5
216 5
428 154
451 4
456 5
459 5
503 16
80 5
87 5
SL2 5
8 5
137 5
14 5
220 5
24 5
351 7
554 5
91 5
99 5
501 5
221 61
431 3
448 139
553 5
69 5
92 5
455 5
68 5
746 5
75 5
5 72
11 5
112 5
16 5
22 3
34 5
430 5
502 4
89 3
90 5
109 4
192 7
194 26
214 5
240 5
245 5
441 5
55 5
77 3
117 5
131 5
17 5
211 5
350 5
36 5
436 5
10 5
105 5
108 5
23 4
29 5
31 5
32 4
352 5
45 5
47 5
93 5
210 66
465 5
505 5
71 5
79 5
59 5
72 5
136 5
504 5
558 5
70 5
86 5
202 5
439 4
50 5
57 3
84 42
19 5
30 5
85 5
CT1 5
101 4
217 5
33 5
450 5
212 5
354 5
70A 5
73 4
94 4
95 5
74 5
429 4
110 5
40 5
556 4
97 4
CT2 5
1 4
27 5
37 4
64 5
18 5
67 5
SL4 4
9703 84
120 5
134 5
15 4
34E 4
608 5
83 5
193 20
38 5
191 6
325 39
41 5
42 5
96 5
28 5
66 4
238 5
9701 54
121 5
9702 37
52 5
195 198

In [74]:
routes = ["1", "7", "15", "22", "23", "28", "32", "39", "57", "66", "71", "73", "77", "100", "111", "116"]
out = pd.DataFrame()
for i in routes:
    out = pd.concat([out, pd.DataFrame(daysSlowerThanAvg[i])])
    
    
out.to_csv('out3.csv')
#daysSlowerThanAvg['1']

**Function to determine average travel times to and from each census tract in the Uber dataset.**

Output of this function has been saved in "times.csv" and the timesDf.

In [13]:
def averageTravelTimes(sources, dests, dataset):
    fromtotimes = {}
    counter = 0
    for source in sources:
        sliced = dataset.loc[(dataset['sourceid'] == source)]
        for dest in dests:
            furthersliced = sliced.loc[(sliced['dstid'] == dest)]
            if len(furthersliced.index != 0):
                mean = furthersliced['geometric_mean_travel_time'].mean()
                fromtotimes[(source, dest)] = mean
                counter+=1
                if (counter % 5000) == 0:
                    print(source, dest, mean)
    return fromtotimes

In [236]:
#times = averageTravelTimes(sources, dests, uber)

In [103]:
#timesSeries.index.name = 'To From Pair'

In [104]:
#timesSeries.reset_index()
#timesSeries.to_csv('times.csv')

In [53]:
timesDf.loc[(timesDf["start"] == 837) & (timesDf["stop"] == 587)]

,start,stop,time
151339,837,587,611.565573


**To compare Uber and MBTA bus lines, we need to know what census tracts each line goes through. Here's the process:**

1. Find a map with census tracts. I use https://worldmap.harvard.edu/maps/3948. Uncheck everything on the left, and then under "Boundaries" check "Boston's Census Tracts".

2. Use the MBTA's website to get a map of the line route. https://www.mbta.com/schedules/bus

3. Click on areas of the Harvard Map that the line you're working on goes through. Write down the census tracts in a list. The Harvard website shows the tract under "Feature Details" and then "TRACT". 

4. Repeat for each line. There's 180 of them, so maybe start with the most popular lines first: https://en.wikipedia.org/wiki/MBTA_key_bus_routes

In [280]:
#Luke doing Corr Analysis on these 4/22/18 10:44am
zones_1 = ["806", "804", "711", "709", "708", "105", "107", "108", "3531", "3530", "3538", "3537"]
zones_7 = ["885", "883", "530", "425", "501"]
zones_15 = ["091500","091200","091300","090600", "080100", "080300", "080400", "080600"]  #not sure why the tract numbers look different on mine - TRF
zones_22 = ["1008", "1005", "1001", "924", "901", "821", "819", "813", "814", "806.01"]

zones_23 = ["1008", "1005", "922", "920", "919", "902", "820", "818", "803", "806.01"]
zones_28 = ["1010.02", "1011.02", "1002", "1001", "924", "901", "902", "820", "818", "803", "806.01"]
zones_32 = ["1402.01", "1403", "1401.05", "1404", "1102.01", "1103.01", "1101.03"]
zones_39 = ["1101.03", "1201.04", "1201.03", "1204", "1207", "811", "810.01", "809", "808.01", "104.05", "105", "106", "703", "702", "701.01", "303"]

zones_57 = ["3704", "3731", "3.01", "2.01", "2.02", "6.01", "8.02", "8.03", "101.03", "101.04", "107.01", "107.02", "106", "703", "702", "701.01", "303"]
zones_66 = ["806.01", "814", "809", "810.01", "811", "9818", "4009", "4008", "4004", "4003", "7.03", "7.04", "6.02", "8.02", "1", "9815.01", "3541", "3540", "3537"]
zones_71 = ["3540", "3537", "3541", "3542", "3543", "3703", "3702.01", "3702.02", "3701.02"]
zones_73 = ["3540", "3537", "3541", "3542", "3543", "3573", "3574", "3575", "3577"]

zones_77 = ["3540", "3545", "3547", "3550", "3567.01", "3567.02", "3566.01", "3566.02", "3565"]
zones_100 = ["837", "588", "838", "836", "203", "587"]
zones_111 = ["303", "302", "203.03", "401", "401.01", "402", "408.01", "1603", "1602", "1604", "1605.01", "1606.01", "1606.02", "1701"]
zones_116 = ["504", "506", "502", "501.01", "509.01", "1603", "1602", "1604", "1601.01", "1605.02", "1706.01", "1704", "1705.01"]
